In [1]:
import torch
import torch.nn.functional as F
from torch.optim import SGD
from torchvision.models import vgg11_bn, VGG11_BN_Weights

/home/yufan/anaconda3/envs/pruning_app/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
from model.build_model import all_classifiers
model = all_classifiers["resnet18"]

In [25]:
from prettytable import PrettyTable

def simplify_number(num):
    if num >= 1_000_000:
        return f"{num / 1_000_000:.2f} M"
    elif num >= 1_000:
        return f"{num / 1_000:.2f} K"
    else:
        return str(num)

# Test the function
# print(simplify_number(28149514))

param_num = {}
layer_spec = {}
total_param = 0
for name, param in model.named_parameters():
    n = name.split(".")[:2]
    n = ".".join(n)
    if n not in param_num:
        param_num[n] = param.numel()
    else:
        param_num[n] += param.numel()
    total_param += param.numel()

table = PrettyTable(["Name", "Param Num"])
for name, num in param_num.items():
    table.add_row([name, simplify_number(num)])
print(table)
print(f"Total Param Num: {simplify_number(total_param)}")

+--------------+-----------+
|     Name     | Param Num |
+--------------+-----------+
|  features.0  |   1.79 K  |
|  features.1  |    128    |
|  features.4  |  73.86 K  |
|  features.5  |    256    |
|  features.8  |  295.17 K |
|  features.9  |    512    |
| features.11  |  590.08 K |
| features.12  |    512    |
| features.15  |   1.18 M  |
| features.16  |   1.02 K  |
| features.18  |   2.36 M  |
| features.19  |   1.02 K  |
| features.22  |   2.36 M  |
| features.23  |   1.02 K  |
| features.25  |   2.36 M  |
| features.26  |   1.02 K  |
| classifier.0 |   2.10 M  |
| classifier.3 |  16.78 M  |
| classifier.6 |  40.97 K  |
+--------------+-----------+
Total Param Num: 28.15 M


In [39]:
layer_spec = {}
spec_tab = PrettyTable(["Name", "Spec"])

for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        layer_spec[name] = module
        spec_tab.add_row([name, f"Conv {module.kernel_size[0]}x{module.kernel_size[1]}, {module.out_channels}"])
    elif isinstance(module, torch.nn.Linear):
        spec_tab.add_row([name, f"Linear, {module.out_features}"])
tab = spec_tab.get_formatted_string()
print(tab)

+-----------------------+---------------+
|          Name         |      Spec     |
+-----------------------+---------------+
|         conv1         |  Conv 3x3, 64 |
|     layer1.0.conv1    |  Conv 3x3, 64 |
|     layer1.0.conv2    |  Conv 3x3, 64 |
|     layer1.1.conv1    |  Conv 3x3, 64 |
|     layer1.1.conv2    |  Conv 3x3, 64 |
|     layer2.0.conv1    | Conv 3x3, 128 |
|     layer2.0.conv2    | Conv 3x3, 128 |
| layer2.0.downsample.0 | Conv 1x1, 128 |
|     layer2.1.conv1    | Conv 3x3, 128 |
|     layer2.1.conv2    | Conv 3x3, 128 |
|     layer3.0.conv1    | Conv 3x3, 256 |
|     layer3.0.conv2    | Conv 3x3, 256 |
| layer3.0.downsample.0 | Conv 1x1, 256 |
|     layer3.1.conv1    | Conv 3x3, 256 |
|     layer3.1.conv2    | Conv 3x3, 256 |
|     layer4.0.conv1    | Conv 3x3, 512 |
|     layer4.0.conv2    | Conv 3x3, 512 |
| layer4.0.downsample.0 | Conv 1x1, 512 |
|     layer4.1.conv1    | Conv 3x3, 512 |
|     layer4.1.conv2    | Conv 3x3, 512 |
|           fc          |   Linear

In [7]:

# To get the parameter count of each layer like Keras, PyTorch has model.named_parameters() that returns an iterator over both the parameter name and the parameter itself. Example:

from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters", ])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(model)

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
|  features.0.weight  |    1728    |
|   features.0.bias   |     64     |
|  features.1.weight  |     64     |
|   features.1.bias   |     64     |
|  features.4.weight  |   73728    |
|   features.4.bias   |    128     |
|  features.5.weight  |    128     |
|   features.5.bias   |    128     |
|  features.8.weight  |   294912   |
|   features.8.bias   |    256     |
|  features.9.weight  |    256     |
|   features.9.bias   |    256     |
|  features.11.weight |   589824   |
|   features.11.bias  |    256     |
|  features.12.weight |    256     |
|   features.12.bias  |    256     |
|  features.15.weight |  1179648   |
|   features.15.bias  |    512     |
|  features.16.weight |    512     |
|   features.16.bias  |    512     |
|  features.18.weight |  2359296   |
|   features.18.bias  |    512     |
|  features.19.weight |    512     |
|   features.19.bias  |    512     |
|

28149514

In [9]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [16]:
import torch
import torchvision
import torchvision.transforms as transforms


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
# define the trainer and evaluaF.nll_loss(output, target, reduction='sum').item()tor
def trainer(model, optimizer, criterion, train_loader):
    # training the model
    model.train()

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        



In [14]:
import torch.nn as nn
def evaluator(model, test_loader):
    # evaluating the model accuracy and average test loss
    model.eval()
    test_loss = 0
    correct = 0
    test_dataset_length = 64
    loss = nn.CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # sum up batch loss
            test_loss += loss(output, target)
            # get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            break
    test_loss /= test_dataset_length
    accuracy = 100. * correct / test_dataset_length
    print('Average test loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(test_loss, correct, test_dataset_length, accuracy))

In [15]:
evaluator(model, testloader)

Average test loss: 0.3423, Accuracy: 0/64 (0%)


In [9]:
config_list = [{
    'op_types': ['Conv2d', "Linear"],
    'exclude_op_names': ['classifier.6'],
    'sparse_ratio': 0.5
}]

In [10]:
from nni.compression.pruning import L1NormPruner, L2NormPruner
pruner = L2NormPruner(model, config_list)

# show the wrapped model structure, `PrunerModuleWrapper` have wrapped the layers that configured in the config_list.
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (_nni_wrapper): ModuleWrapper(module=Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), module_name=features.0)
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (_nni_wrapper): ModuleWrapper(module=Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), module_name=features.4)
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(
      128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (_nni_wrapper): ModuleWrapper(modul

In [11]:
# compress the model and generate the masks
_, masks = pruner.compress()
# show the masks sparsity
for name, mask in masks.items():
    print(name, ' sparsity : ', '{:.2}'.format(mask['weight'].sum() / mask['weight'].numel()))

features.11  sparsity :  0.5
features.0  sparsity :  0.5
features.18  sparsity :  0.5
features.22  sparsity :  0.5
features.4  sparsity :  0.5
features.8  sparsity :  0.5
features.15  sparsity :  0.5
features.25  sparsity :  0.5
classifier.3  sparsity :  0.5
classifier.0  sparsity :  0.5


In [12]:
# need to unwrap the model, if the model is wrapped before speedup
pruner.unwrap_model()

# speedup the model, for more information about speedup, please refer :doc:`pruning_speedup`.
from nni.compression.speedup import ModelSpeedup

ModelSpeedup(model, torch.rand(64, 3, 32, 32).to(device), masks).speedup_model()

[2023-09-18 14:12:25] Start to speedup the model...
[2023-09-18 14:12:25] Resolve the mask conflict before mask propagate...
[2023-09-18 14:12:25] dim0 sparsity: 0.500000
[2023-09-18 14:12:25] dim1 sparsity: 0.000000
0 Filter
[2023-09-18 14:12:25] dim0 sparsity: 0.500000
[2023-09-18 14:12:25] dim1 sparsity: 0.000000
[2023-09-18 14:12:25] Infer module masks...
[2023-09-18 14:12:25] Propagate original variables
[2023-09-18 14:12:25] Propagate variables for placeholder: x, output mask:  0.0000 
[2023-09-18 14:12:25] Propagate variables for call_module: features_0, weight:  0.5000 bias:  0.5000 , output mask:  0.0000 
[2023-09-18 14:12:25] Propagate variables for call_module: features_1, , output mask:  0.0000 
[2023-09-18 14:12:25] Propagate variables for call_module: features_2, , output mask:  0.0000 
[2023-09-18 14:12:25] Propagate variables for call_module: features_3, , output mask:  0.0000 
[2023-09-18 14:12:25] Propagate variables for call_module: features_4, weight:  0.5000 bias: 

VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kerne

In [13]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kerne

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), 1e-2)
for epoch in range(17):
    trainer(model, optimizer, criterion, trainloader)
    

In [21]:
evaluator(model, testloader)

Average test loss: 0.0092, Accuracy: 53/64 (83%)
